In [ ]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int

from utils import print_shape, print_name
from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.random_fourier_features import RandomFourierFeatures

jax.config.update('jax_platform_name', 'cpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

def gen_BM(N:int, T:int, D:int, seed=0):
    """
    Generate N Brownian Motions of length T with D dimensions
    """
    key = jax.random.PRNGKey(seed)
    normal = jax.random.normal(key, (N, T, D))
    BM = jnp.cumsum(normal, axis=1)
    return BM

In [ ]:
N=10
D=100
rff_dim = 1020
seed = jax.random.PRNGKey(9999)
X = jax.random.normal(seed, (N, D))

rff = RandomFourierFeatures(seed, n_features=rff_dim, sigma=1.0, max_batch=2000)
rff.fit(X)
feat = rff.transform(X)
print_name(feat)

In [ ]:
N=200
T=1000
D=10
X = gen_BM(N, T, D)
seed = jax.random.PRNGKey(0)

max_batch=10
trunc_level=5
n_features=500

linear_trp = SigVanillaTensorizedRandProj(seed, n_features, trunc_level, max_batch)
linear_trp.fit(X)
feat = linear_trp.transform(X)
print_name(feat)

In [ ]:
N=100
max_batch = 22


T=100
D=10
X = gen_BM(N, T, D)
seed = jax.random.PRNGKey(0)
trp_seed, rff_seed = jax.random.split(seed, 2)
n_features = 2000
trunc_level = 3
rbf_dimension = 512
sigma = 1.0
rff_max_batch = 2000

rbf_trp = SigRBFTensorizedRandProj(trp_seed, rff_seed, 
                n_features, trunc_level, rbf_dimension, 
                sigma, max_batch, rff_max_batch)
rbf_trp.fit(X)

print("now\n\n\n")
feat = rbf_trp.transform(X)
print_name(feat)